In [68]:
import pandas as pd
import numpy as np
import pickle 
from tqdm import tqdm 
from collections import Counter,OrderedDict
from sklearn.metrics.pairwise import cosine_similarity

def flatten_comprehension(matrix):
    #https://realpython.com/python-flatten-list/
    return [item for row in matrix for item in row]

# Load data
behaviors = pd.read_csv('./small_training_data/behaviors.tsv', delimiter='\t', header=None)
news = pd.read_csv('./small_training_data/news.tsv', delimiter='\t', header=None)

# Naming columns
behaviors.columns = ["impression_id", "user_id", "time", "history", "impressions"]
news.columns = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]


# Extracting clicked news from behaviors, this is a column of lists of the clicked news (tagget with 1) for each impression
behaviors['clicked_news'] = behaviors['impressions'].apply(lambda x: [imp.split('-')[0] for imp in x.split() if imp.split('-')[1] == '1'])

In [14]:
#define unique users and their read articles
unique_users_series = behaviors["user_id"].drop_duplicates()
unique_users_dict = {}  #user_id coupled to news_articles

#aggregate all read articles and couple it to the user in the unique_users_frame
i = 0
for user_id in tqdm(unique_users_series,ascii=True):
    user_specific_behavior = behaviors[behaviors["user_id"] == user_id]
    try:
        history_list = flatten_comprehension([elem.split(" ") for elem in user_specific_behavior["history"]])
    except:
        pass 
    clicked_list = flatten_comprehension([elem for elem in user_specific_behavior["clicked_news"]])
    all_read_articles = history_list + clicked_list
    unique_users_dict[user_id] = all_read_articles


with open('unique_users.pkl', 'wb') as file: #serialize so we don't have to do this inefficient code again
    pickle.dump(unique_users_dict,file)



100%|#################################################| 50000/50000 [04:24<00:00, 189.29it/s]


In [74]:
#define the embedding. (what words we look for)

total_title = ""
for title in tqdm(news["title"],ascii=True):
    total_title += " "+title 

total_BOW = Counter(total_title.lower().split(" "))
sorted_BOW = sorted(OrderedDict(total_BOW).items(),key=lambda x: x[1],reverse=True)



    

100%|################################################################################################################################################| 51282/51282 [00:11<00:00, 4294.03it/s]


In [82]:
embedding_words = [elem[0] for elem in sorted_BOW[30:500]]

with open('unique_users.pkl', 'rb') as file:
    users = pickle.load(file)

user_BOW = {} #{"user_id":embedding,etc.}

#compute user embeddings
for user in tqdm(users,ascii=True):
    news_ids = users[user]     
    user_BOW[user] = Counter(news["title"][news["news_id"].isin(news_ids)].map(str.lower).apply(lambda x: x +" ").agg("sum"))
    print(user_BOW)
    break


# with open('userBOW.pkl', 'wb') as file: #serialize so we don't have to do this inefficient code again
#     pickle.dump(user_BOW,file)

100%|#################################################################################################################################################| 50000/50000 [01:09<00:00, 723.13it/s]


In [84]:
with open('userBOW.pkl', 'rb') as file:
    user_BOW = pickle.load(file)

ordered_userBOW = OrderedDict(user_BOW)



In [ ]:
"""
NOTE 

instead of cosine similarity we use Jaccard similarity J (https://stats.stackexchange.com/questions/289400/quantify-the-similarity-of-bags-of-words)
because the BOW embeddings are really sparse except if you use to common words ( "a", "is", "of",etc) 
This would be counter intuitive to use. 

"""